In [24]:
import transformers
from ultralytics import YOLO
from PIL import Image, ImageOps

In [14]:
text_detector = YOLO("../../models/text_tedector/text_detector.pt").to("cpu")

In [29]:
img = Image.open(r"P:\PyProjects\WORK_PROJECTS\RANEPA\Historical-docs-OCR\data\raw\Распознавание текстов\Уставные грамоты в jpg (Просветов)\11227024_doc1.jpg")
img = ImageOps.exif_transpose(img)

res = text_detector.predict([img])

# Process results list
for result in res:
    boxes = result.boxes  # Boxes object for bounding box outputs
    masks = result.masks  # Masks object for segmentation masks outputs
    keypoints = result.keypoints  # Keypoints object for pose outputs
    probs = result.probs  # Probs object for classification outputs
    result.show()  # display to screen

0: 640x448 36 texts, 364.3ms
Speed: 1.0ms preprocess, 364.3ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 448)


In [34]:
from datasets import load_metric

cer_metric = load_metric("cer", trust_remote_code=True)
wer_metric = load_metric("wer", trust_remote_code=True)

In [46]:
predictions = ["this is the prediction", "there is an other sample"]
references = ["this is the reference", "there is another one"]

cer_score = cer_metric.compute(predictions=predictions, references=references)
wer_score = wer_metric.compute(predictions=predictions, references=references)

print(cer_score, wer_score)

0.34146341463414637 0.5
